## Image generation

In [ ]:
%pip install diffusers==0.3.0
%pip install transformers scipy ftfy
%pip install "ipywidgets>=7,<8"

from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from diffusers import StableDiffusionPipeline

# make sure you're logged in with `huggingface-cli login`
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=True)
pipe.safety_checker = lambda images, clip_input: (images, False) # remove safety checker
pipe = pipe.to("cuda")

In [ ]:
from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid


In [ ]:
from torch import autocast

n_examples = 5

for i in range(n_examples):
  book_idx = random.randint(0, len(books)-1)
  book = books[book_idx]
  book_url = book['book_url']
  pairs = list(zip(book['book_img'], book['book_txt']))
  pair_idx = random.randint(0, len(pairs)-1)
  img, txt = pairs[pair_idx]

  print(f'\nExample {i}: book {book_idx} ({book_url}), paragraph {pair_idx}')
  print('\tReference text:', txt)
  urllib.request.urlretrieve(img, f"{i}.png")
  Img = Image.open(f"{i}.png")
  print('\tReference image:')
  display(Img)

  prompt = txt
  with autocast("cuda"):
    gen_image = pipe(prompt + ", colorful children's book illustration.").images[0]
  print('\tGenerated image:')
  display(gen_image)